In [1]:
import numpy as np
import SimpleITK as sitk
import radiomics
import six
from radiomics import featureextractor
from PIL import Image

# copy image locally
print("Opening original image")
img  = Image.open('/data/athena_screen/images/00a560c1095b7035d62627db4053a4b9.png')  
width, height = img.size
print("width=%s, height=%s" % (width, height))
img.save("/home/willhsu/tmp_image.png") 

# load image using SimpleITK
reader = sitk.ImageFileReader()
reader.SetImageIO("PNGImageIO")
reader.SetFileName("/home/willhsu/tmp_image.png")
image2d = reader.Execute();

# create 3D version of image
image3d = sitk.JoinSeries(image2d)

Opening original image
width=2560, height=3328


In [2]:
# generate mask to match dimensions of original image
print("Generating mask")
mask = np.zeros((height,width),dtype=np.uint8)
mask.fill(1) # or img[:] = 255
im = Image.fromarray(mask)
width, height = im.size
print("width=%s, height=%s" % (width, height))
im.save("/home/willhsu/tmp_mask.png")

# load image using SimpleITK
reader = sitk.ImageFileReader()
reader.SetImageIO("PNGImageIO")
reader.SetFileName("/home/willhsu/tmp_mask.png")
image2d = reader.Execute();

# create 3D version of image
mask3d = sitk.JoinSeries(image2d)

Generating mask
width=2560, height=3328


In [3]:
# calculate (first-order) radiomic features using pyradiomics
print("Calculating features")
extractor = featureextractor.RadiomicsFeaturesExtractor()
extractor.disableAllFeatures()
extractor.enableFeatureClassByName('firstorder')
extractor.enableFeatureClassByName('glcm')
extractor.enableFeatureClassByName('glszm')
extractor.enableFeatureClassByName('glrlm')
featureVector = extractor.execute(image3d, mask3d)

for (key,val) in six.iteritems(featureVector ):
  print("\t%s: %s" % (key, val))

Calculating features


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


	diagnostics_Versions_PyRadiomics: 2.1.2
	diagnostics_Versions_Numpy: 1.16.2
	diagnostics_Versions_SimpleITK: 1.2.0
	diagnostics_Versions_PyWavelet: 1.0.0
	diagnostics_Versions_Python: 3.5.2
	diagnostics_Configuration_Settings: {'distances': [1], 'padDistance': 5, 'resampledPixelSpacing': None, 'resegmentRange': None, 'normalize': False, 'minimumROIDimensions': 2, 'preCrop': False, 'normalizeScale': 1, 'label': 1, 'removeOutliers': None, 'minimumROISize': None, 'force2Ddimension': 0, 'additionalInfo': True, 'force2D': False, 'interpolator': 'sitkBSpline'}
	diagnostics_Configuration_EnabledImageTypes: {'Original': {}}
	diagnostics_Image-original_Hash: 2c8ecf0c88d31ae10e804fbc17277e8fc2f4f096
	diagnostics_Image-original_Spacing: (1.0, 1.0, 1.0)
	diagnostics_Image-original_Size: (2560, 3328, 1)
	diagnostics_Image-original_Mean: 25.313341111403247
	diagnostics_Image-original_Minimum: 0.0
	diagnostics_Image-original_Maximum: 255.0
	diagnostics_Mask-original_Hash: 71e4d958584d18520777fb43993

In [4]:
#!ls /data/athena_screen/images